In [ ]:
import functools
import logging
import re
import traceback

import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType
from pyspark.sql.utils import AnalysisException

# Custom Exceptions

In [ ]:
def analyze_analysis_exception(exception: AnalysisException) -> dict:
    """Analyzes a PySpark AnalysisException to extract details.

    This function parses the error message from a PySpark `AnalysisException` to identify common
    causes of Spark SQL failures, such as missing columns, missing tables, ambiguous columns,
    missing paths, data type mismatches, and schema mismatches. It returns a dictionary with
    standardized fields for error type, a short name, a human-readable description, extracted
    details, the raw error message, and the stacktrace.

    Args:
        exception (AnalysisException): The PySpark AnalysisException instance to analyze.

    Returns:
        dict: A dictionary containing:
            - error_type (str): A standardized error category (e.g., 'MissingColumnError').
            - short_name (str): A short, code-friendly error name (e.g., 'ColumnNotFound').
            - description (str): A human-readable description of the error.
            - details (dict): Extracted details relevant to the error type (e.g., column or table names).
            - raw_message (str): The original error message from the exception.
            - stacktrace (str): The stacktrace at the point of exception analysis.

    Example:
        try:
            df.select("non_existent_column").show()
        except AnalysisException as e:
            info = analyze_analysis_exception(e)
            print(info["description"])
            # Output: "Column 'non_existent_column' not found in DataFrame"

    Notes:
        - The function uses regex to extract specific error details from known AnalysisException patterns.
        - If the error does not match a known pattern, a generic error type and description are returned.
        - This function is intended to support robust error logging and user-friendly diagnostics in
          data quality and validation workflows.
    """
    error_message = str(exception)
    result = {
        "error_type": "UnknownAnalysisError",
        "short_name": "AnalysisException",
        "description": "An error occurred during Spark SQL analysis",
        "details": {},
        "raw_message": error_message,
        "stacktrace": traceback.format_exc()
    }
    
    # Extract column name from "cannot resolve '`column_name`' given input columns: [col1, col2, ...]"
    if "cannot resolve" in error_message and "given input columns" in error_message:
        result["error_type"] = "MissingColumnError"
        result["short_name"] = "ColumnNotFound"
        
        # Extract the unresolved column name
        unresolved_match = re.search(r"cannot resolve '(?:`)?([^'`]+)(?:`)?'", error_message)
        unresolved_column = unresolved_match.group(1) if unresolved_match else "unknown"
        
        # Extract available columns
        available_match = re.search(r"given input columns: \[(.*?)\]", error_message)
        available_columns = available_match.group(1).split(", ") if available_match else []
        
        result["description"] = f"Column '{unresolved_column}' not found in DataFrame"
        result["details"] = {
            "unresolved_column": unresolved_column,
            "available_columns": available_columns
        }
    
    # Extract table name from "Table or view not found: `table_name`"
    elif "Table or view not found" in error_message:
        result["error_type"] = "MissingTableError"
        result["short_name"] = "TableNotFound"
        
        # Extract the missing table name
        table_match = re.search(r"Table or view not found: (?:`)?([^'`]+)(?:`)?", error_message)
        table_name = table_match.group(1) if table_match else "unknown"
        
        result["description"] = f"Table or view '{table_name}' not found"
        result["details"] = {
            "missing_table": table_name
        }
    
    # Handle ambiguous column references
    elif "Ambiguous column reference" in error_message or "Ambiguous column name" in error_message:
        result["error_type"] = "AmbiguousColumnError"
        result["short_name"] = "AmbiguousColumn"
        
        # Extract the ambiguous column name
        col_match = re.search(r"[Aa]mbiguous column (?:reference|name)(?:: | ')([^']+)", error_message)
        col_name = col_match.group(1) if col_match else "unknown"
        
        result["description"] = f"Column '{col_name}' is ambiguous (exists in multiple tables/sources)"
        result["details"] = {
            "ambiguous_column": col_name
        }
    
    # Handle file/path not found errors
    elif "Path does not exist" in error_message:
        result["error_type"] = "PathNotFoundError"
        result["short_name"] = "PathNotFound"
        
        # Extract the missing path
        path_match = re.search(r"Path does not exist: ([^']+)", error_message)
        path = path_match.group(1) if path_match else "unknown"
        
        result["description"] = f"Path '{path}' does not exist"
        result["details"] = {
            "missing_path": path
        }
    
    # Handle data type mismatch errors
    elif "data type mismatch" in error_message.lower():
        result["error_type"] = "DataTypeMismatchError"
        result["short_name"] = "DataTypeMismatch"
        result["description"] = "Data type mismatch in operation"
        
        # Try to extract more details if available
        type_match = re.search(r"CAST\(([^)]+) AS ([^)]+)\)", error_message)
        if type_match:
            result["details"] = {
                "from_type": type_match.group(1),
                "to_type": type_match.group(2)
            }
    
    # Handle schema mismatch errors
    elif "schema mismatch" in error_message.lower():
        result["error_type"] = "SchemaMismatchError"
        result["short_name"] = "SchemaMismatch"
        result["description"] = "Schema mismatch between operations"
    
    return result

# Decorators

In [ ]:
def add_try_except(func):
    """Decorator providing robust exception handling for class methods.

    This decorator wraps a method and intercepts common exceptions that may occur during
    data quality operations, including:
        - KeyError: Raised when a dictionary key is missing.
        - IndexError: Raised when a list or sequence index is out of bounds.
        - AnalysisException: Raised by PySpark SQL for issues such as missing columns, tables, or schema mismatches.
        - General Exception: Catches all other unexpected exceptions.

    For each exception, the decorator:
        - Logs the error message to self._logger if a logger is present.
        - Prints the error message to standard output.
        - Appends a structured error entry to self._row_errors (if available), including error type, method name, message, and details.

    If an exception is caught, the method returns None by default.

    Args:
        func (callable): The instance method to be wrapped.

    Returns:
        callable: The wrapped method with exception handling.

    Example:
        @add_try_except
        def log_processed_file(self, source_data_df: DataFrame, success: bool) -> None:
            # Method implementation...

    Notes:
        - This decorator assumes the wrapped method is an instance method with 'self' as its first argument.
        - The decorator should be defined before the class and applied to any method where robust error handling is desired.
        - If VERBOSE_LOGGING is enabled on the instance, additional details and stacktraces are logged for general exceptions and AnalysisException.
        - The decorator is designed for use in data quality, validation, and ETL workflows where consistent error handling and logging are required.
    """
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        try:
            return func(self, *args, **kwargs)
        except KeyError as e:
            error_msg = f"KeyError in {func.__name__}: {str(e)}"
            if hasattr(self, 'logger') and self._logger:
                self._logger.error(error_msg)
            print(error_msg)
            if hasattr(self, 'error_logs'):
                self._row_errors.append({
                    "error_type": "KeyError",
                    "method": func.__name__,
                    "message": str(e)
                })
            return None
            
        except IndexError as e:
            error_msg = f"IndexError in {func.__name__}: {str(e)}"
            if hasattr(self, 'logger') and self._logger:
                self._logger.error(error_msg)
            print(error_msg)
            if hasattr(self, 'error_logs'):
                self._row_errors.append({
                    "error_type": "IndexError",
                    "method": func.__name__,
                    "message": str(e)
                })
            return None
            
        except AnalysisException as e:
            # Use the analysis function for PySpark errors
            error_info = analyze_analysis_exception(e)
            error_msg = f"AnalysisException in {func.__name__}: {error_info['description']}"
            
            if hasattr(self, 'logger') and self._logger:
                self._logger.error(error_msg)
                if hasattr(self, 'VERBOSE_LOGGING') and self._verbose_logging:
                    self._logger.error(f"Details: {error_info['details']}")
            
            print(error_msg)
            
            if hasattr(self, 'error_logs'):
                self._row_errors.append({
                    "error_type": error_info['error_type'],
                    "method": func.__name__,
                    "message": error_info['description'],
                    "details": error_info['details'] if 'details' in error_info else {}
                })
            return None
            
        except Exception as e:
            error_msg = f"Exception in {func.__name__}: {str(e)}"
            if hasattr(self, 'logger') and self._logger:
                self._logger.error(error_msg)
                if hasattr(self, 'VERBOSE_LOGGING') and self._verbose_logging:
                    self._logger.error(traceback.format_exc())
            print(error_msg)
            if hasattr(self, 'error_logs'):
                self._row_errors.append({
                    "error_type": "GeneralException",
                    "method": func.__name__,
                    "message": str(e),
                    "stacktrace": traceback.format_exc() if hasattr(self, 'VERBOSE_LOGGING') and self._verbose_logging else None
                })
            return None
    
    return wrapper

In [ ]:
def log_method_call(stage: str):
    """
    Decorator to log method calls in a class.

    Each time the decorated method is called, a dictionary is appended to the instance's `method_log` list.
    The log entry contains:
        - "stage": A string indicating the processing stage (e.g., "file_validation", "row_processing").
        - "method": The name of the method being called.
        - "parameters": A dictionary of the positional and keyword arguments supplied to the method.
        - "status": "success" if the method completes without exception, "failure" otherwise.

    Args:
        stage (str): The processing stage to associate with this method.

    Usage:
        @log_method_call("file_validation")
        def file_validation(self, ...):
            ...

    The class using this decorator should have a `method_log` attribute (a list) to store log entries.
    """
    def decorator(func):
        def wrapper(self, *args, **kwargs):
            log_entry = {
                "stage": stage,
                "method": func.__name__,
                "parameters": {"args": args, "kwargs": kwargs},
                "status": "success"
            }
            try:
                result = func(self, *args, **kwargs)
                return result
            except Exception as e:
                log_entry["status"] = "failure"
                raise
            finally:
                self._method_calls.append(log_entry)
        return wrapper
    return decorator

# Data Quality Class

In [ ]:
class DataQualityLibrary:
    """Provides a suite of methods for validating data quality, transforming data and logging errors in Spark-based data pipelines.

    Features:
    - File and field-level validation using configurable rules.
    - Logging of processed files and failed rows for traceability.
    - Error handling and logging with support for verbose output.
    - Conversion of logs to Spark DataFrames for further analysis or persistence.
    - Writing error logs to Delta tables in a Lakehouse architecture.
    - Decorator-based robust error handling for all methods.

    Args:
        spark: The active SparkSession.
        run_id (str): Unique identifier for the current pipeline run.
        trigger_time (str): Timestamp or identifier for the pipeline trigger event.
        verbose_logging (bool, optional): Enable detailed logging. Default is False.
        logger (optional): Logger instance for custom logging. If None, uses default logging.

    Example usage:
        dq = DataQualityLibrary(spark, run_id="123", trigger_time="2025-09-05", verbose_logging=True, logger=my_logger)
        dq.file_validation(df, file_rules, "myfile.csv")
        dq.field_validation(df, field_rules, file_rules, "table_name")
    """
    def __init__(self, spark, run_id: str, trigger_time: str, verbose_logging=False, logger=None) -> None:
        self._row_errors = []
        self._method_calls = []
        self._verbose_logging = verbose_logging
        self._logger = logger
        self._spark = spark
        self._run_id = run_id
        self._trigger_time = trigger_time
        self._processed_files = []
        self._initial_rowcount = None
    
    @add_try_except
    def _get_lookup_table(self, table_name: str) -> DataFrame:
        """Use a Fabric lakehouse shortcut to retrieve a lookup table as a Spark DataFrame."""
        lookup_df = self._spark.table(table_name)
        
        return lookup_df

    @add_try_except
    def toggle_verbose_logging(self) -> None:
        """Toggles the verbose logging flag."""
        self._verbose_logging = not self._verbose_logging

    @add_try_except
    def processed_files_to_df(self) -> DataFrame:
        """Converts the processed_files list to a Spark DataFrame."""
        return self._spark.createDataFrame(self._processed_files)

    @add_try_except
    def reset_error_logs(self):
        """Resets the error_logs property to an empty list."""
        self._row_errors = []

    @add_try_except
    def row_errors_to_df(self) -> DataFrame:
        """Converts the error_logs list to a Spark DataFrame."""
        return self._spark.createDataFrame(self._row_errors)
    
    @add_try_except
    def method_calls_to_df(self) -> DataFrame:
        """Converts the method_calls list to a Spark DataFrame."""
        return self._spark.createDataFrame(self._method_calls)

    @add_try_except
    def _log_failed_rows(self, data_df: DataFrame, validation: str, rule_name: str, validation_stage: str) -> DataFrame:
        """Logs rows which have failed validation."""
        # Get rows which contain an error message
        failed_rows_df = data_df.where(F.col("error_message").isNotNull())
        # Create a log entry for each row which failed validation
        for row in failed_rows_df.collect():
            log = {
                "datasource": row.datasource,
                "entity_name": row.entity_name,
                "row_id": row.row_id,
                "validation": validation,
                "rule_name":rule_name,
                "error_message": row.error_message,
                "stage": validation_stage
            }
            self._row_errors.append(log)
            if self._verbose_logging:
                self._logger.warning("datasource '%s' entity '%s' row '%s' failed with error in %s: %s. Error: %s", row.datasource, row.entity_name, row.row_id, validation, rule_name)
        # Drop error message column
        data_df = data_df.drop("error_message")

        return data_df
    
    # File-level validation functions
    @add_try_except
    def file_validation(self, source_data_df: DataFrame, metadata_rules: dict, source_filepath: str) -> bool:
        """Function to validate the columns of the source file.
        The function validates that all mandatory columns are present, and that no extraneous columns are
        present in the source file.
        Args:
            source_data_df (DataFrame): DataFrame containing data from the source file.
            metadata_rules (dict): Dictionary containing the metadata from file_<datasource>.yaml.
            source_filepath (str): The name of the file being validated.
        Returns:
            Bool: Validation results.
            source_data_df (DataFrame): DataFrame potentially modified to add missing optional columns or drop ignored columns.
        """
        # Lists for column comparison.
        all_metadata_columns = []
        all_df_columns = source_data_df.columns
        # Read in columns metadata
        mandatory_columns = metadata_rules.get("mandatory_columns")
        optional_columns = metadata_rules.get("optional_columns", [])
        ignore_columns = metadata_rules.get("ignore_columns", [])
        for item in mandatory_columns:
            column_name = item["name"]
            all_metadata_columns.append(column_name)
            # Check that the mandatory column exists in source
            if not column_name in all_df_columns:
                if self._verbose_logging:
                    self._logger.warning(f"File {source_filepath} does not contain the mandatory column: {column_name}")
                return False, source_data_df
        # Add optional columns to all_metadata_columns
        for item in optional_columns:
            column_name = item["name"]
            all_metadata_columns.append(column_name)
            # Add the optional column if it doesn't exist in source df
            if not column_name in source_data_df.columns:
                source_data_df = source_data_df.withColumn(column_name, F.lit(None))
        # Add ignore columns all_metadata_columns
        for item in ignore_columns:
            ignore_col = item["name"]
            if ignore_col in source_data_df.columns:
                source_data_df = source_data_df.drop(ignore_col)
            all_metadata_columns.append(ignore_col)
        # Check that no extraneous columns are in source data
        is_all_columns = set(all_df_columns).issubset(set(all_metadata_columns))
        if not is_all_columns:
            if self._verbose_logging:
                self._logger.warning(f"File {source_filepath} contains unexpected columns")
            return False, source_data_df
        return True, source_data_df

    # Field-level validation functions
    @add_try_except
    @log_method_call("field_validation")
    def field_validation(self, source_data_df: DataFrame, metadata_rules: dict) -> None:
        """Main function to do the field validations for a given entity.
        Args:
            source_data_df (DataFrame): Source dataframe 
            metadata_rules (dict): Dictionary containing rules to do the validation. This dictionary is generated from the file field_validation_datasource.yaml
        
        Returns:
            None
        """
        rules = metadata_rules.get("rules")

        for rule in rules:
            processing_type = rule.get("type")
            rule_name = rule.get("name")
            custom_error_code = rule.get("custom_error_code", None)

            match processing_type:
                case "expect_column_values_to_not_be_null":
                    columns_to_validate = rule.get("columns")
                    conditions = rule.get("conditions", None)

                    self._expect_column_values_to_not_be_null(null_check_df=source_data_df,
                                                              columns=columns_to_validate,
                                                              custom_error_message=custom_error_code,
                                                              rule_name=rule_name,
                                                              conditions=conditions)
                            
                case "expect_column_distinct_values_to_be_in_set":
                    columns_to_validate = rule.get("columns")
                    values = rule.get("values")

                    self._expect_column_values_in_set(source_data_df=source_data_df,
                                                      columns=columns_to_validate,
                                                      custom_error_message=custom_error_code,
                                                      value_set=values,
                                                      rule_name=rule_name)     

                case "expect_column_values_to_be_null":
                    columns_to_validate = rule.get("columns")

                    self._expect_column_values_to_be_null(source_data_df=source_data_df,
                                                          columns=columns_to_validate,
                                                          custom_error_message=custom_error_code,
                                                          rule_name=rule_name)
                    
                case "expect_at_least_one_non_null_column":
                    columns_to_validate = rule.get("columns")
                    conditions = rule.get("conditions", None)

                    self._expect_at_least_one_non_null_column(one_non_null_df=source_data_df,
                                                              columns=columns_to_validate,
                                                              custom_error_message=custom_error_code,
                                                              rule_name=rule_name,
                                                              conditions=conditions)
                    
                case "expect_relative_date":
                    time_type = rule.get("time")
                    columns_to_validate = rule.get("columns")

                    self._expect_relative_date(source_data_df=source_data_df,
                                               columns=columns_to_validate,
                                               custom_error_message=custom_error_code,
                                               time_check_type=time_type,
                                               rule_name=rule_name)
                    
                case "expect_column_values_to_be_between":
                    columns_to_validate = rule.get("columns")
                    min_value, max_value = rule.get("min_value", None), rule.get("max_value", None)
                    strict_min, strict_max = rule.get("strict_min", False), rule.get("strict_max", False)

                    self._expect_column_values_between(source_data_df=source_data_df,
                                                       columns=columns_to_validate,
                                                       min_value=min_value,
                                                       max_value=max_value,
                                                       strict_min=strict_min,
                                                       strict_max=strict_max,
                                                       custom_error_message=custom_error_code,
                                                       rule_name=rule_name)
                    
                case "expect_format":
                    columns_to_validate = rule.get("columns")
                    format = rule.get("format", None)
                    length = rule.get("length", None)

                    self._expect_format(source_data_df=source_data_df,
                                        columns=columns_to_validate,
                                        format=format,
                                        length=length,
                                        custom_error_message=custom_error_code,
                                        rule_name=rule_name)
                
                case _:
                    if self._verbose_logging:
                        self._logger.warning(f"Unknown processing type: {processing_type} in rule: {rule_name}")
        
        length_validations = [item for item in metadata_rules["mandatory_columns"] if "max_length" in item]
        self._column_length_checks(source_data_df=source_data_df,
                                   length_check_validations=length_validations,
                                   rule_name="field_length")

        return

    @add_try_except
    def _expect_column_values_between(self, source_data_df: DataFrame, columns: list, min_value: int, max_value: int, strict_min: bool, strict_max: bool, custom_error_message: str, rule_name:str) -> None:
        """Checks column values are between two supplied values.
        Checks a Spark DataFrame to validate that values in the specified column name are between a minimum and maximum value.
        min_value and max_value are both inclusive unless strict_min or strict_max are set to True. If min_value is None, then max_value
        is treated as an upper bound, and there is no minimum value checked. If max_value is None, then min_value is treated as a lower bound, and
        there is no maximum value checked. By default, null values are not considered in computation and so will be handled by separate checks.
        Args:
            source_data_df (DataFrame): Spark DataFrame to validate
            columns (list): Name of the column to validate.
            min_value (int): Minimum allowed column value.
            max_value (int): Maximum allowed column value.
            strict_min (bool): Only allow > comparison as opposed to >=.
            strict_max (bool): Only allow < comparison as opposed to <=.
            custom_error_message (str): Custom error message if provided in the metadata.
            rule_name (str): Name of the metadata rule that called this function.

        Returns:
            None
        """
        validation = "Expect Between"
        for column in columns:
            # Cast column to int for comparison
            source_data_df = source_data_df.withColumn(f"{column}_int", F.col(column).cast("int"))

            # Check for non-integer values and log error if found
            source_data_df = source_data_df.withColumn(
                "error_message",
                F.when(
                    (F.col(column).isNotNull()) & (F.col(f"{column}_int").isNull()),
                    F.concat(
                        F.lit(f"Column '{column}' with value '"),
                        F.col(column),
                        F.lit("' must be an integer")
                    )
                ).otherwise(F.lit(None))
            )

            # Replace original column with casted integer column for further checks
            source_data_df.drop(f"{column}_int")

            # Both min and max provided → BETWEEN check
            if min_value is not None and max_value is not None:
                message = custom_error_message if custom_error_message else f"{column} must be between {min_value} and {max_value}"
                if strict_min and strict_max:
                    condition = (F.col(column) <= min_value) | (F.col(column) >= max_value)
                elif strict_min:
                    condition = (F.col(column) <= min_value) | (F.col(column) > max_value)
                elif strict_max:
                    condition = (F.col(column) < min_value) | (F.col(column) >= max_value)
                else:
                    condition = (F.col(column) < min_value) | (F.col(column) > max_value)
                
                error_message = F.when(
                    condition & F.col("error_message").isNull(),
                    message
                ).otherwise(F.col("error_message"))            
            # Only min provided
            elif min_value is not None:
                message = custom_error_message if custom_error_message else f"{column} must be greater than {min_value}."
                if strict_min:
                    condition = F.col(column) <= min_value
                else:
                    condition = F.col(column) < min_value

                error_message = F.when(
                    condition & F.col("error_message").isNull(),
                    message
                ).otherwise(F.col("error_message"))
            # Only max provided
            elif max_value is not None:
                message = custom_error_message if custom_error_message else f"{column} must be less than {max_value}."
                if strict_max:
                    condition = F.col(column) >= max_value
                else:
                    condition = F.col(column) > max_value
                error_message = F.when(
                    condition & F.col("error_message").isNull(),
                    message
                ).otherwise(F.col("error_message"))

            self._log_failed_rows(data_df=source_data_df,
                                  validation=validation,
                                  rule_name=rule_name,
                                  validation_stage="field_validation")
        
        return
    
    @add_try_except
    def _expect_column_values_in_set(self, source_data_df: DataFrame, columns: list, custom_error_message: str, value_set: list, rule_name:str) -> None:
        """Function to validate if a value in a column is in a given value_set. The validation is run one column at a time and append the 
        failed rows to a global error list.

        Args:
            source_data_df (DataFrame): Spark DataFrame to validate.
            columns (list): List of column(s) in the DataFrame to be validated.
            custom_error_message (str): Custom error message to use.
            value_set (list): List of the expected value that the given column value should be in.
            rule_name (str): Name of the metadata rule that called this function.
            
        Returns:
            None
        """
        validation="Values in Set"

        for column in columns:        
            error_message = custom_error_message if custom_error_message else f"Column {column} contains a value that is not in the list: {value_set}"

            # Get rows which do not have a value in the value_set
            errors_df = source_data_df.where(~F.col(column).isin(value_set))

            # Add error message column
            errors_df = errors_df.withColumn("error_message", F.lit(error_message))

            # Log the failed rows
            self._log_failed_rows(errors_df, validation, rule_name, "field_validation")        
        
        return
    
    @add_try_except
    def _expect_column_values_to_be_null(self, null_check_df: DataFrame, columns: list, custom_error_message: str, rule_name: str, conditions: list = None):
        """Validates if a column is not null.
        Validate if a column is not NULL. If any column in the columns list is Null, flag it as an 
        error and add the custom_error_message. If supplied, the optional conditions parameter is 
        used to filter the dataframe before validation, ensuring the check is only applied to a 
        specific subset of data.

        Args:
            null_check_df (DataFrame): Spark DataFrame to validate
            columns (list[str]): List of column(s) in the DataFrame to be validated
            custom_error_message (str): Custom error message to use.
            rule_name (str): Name of the metadata rule that called this function
            conditions (list[dict]): List of dictionaries which define conditions on which to apply the check.

        Returns:
            None
        """
        validation = "Expect Not Null"

        if conditions:
            for condition in conditions:
                column = condition.get("column")
                comparison = condition.get("comparison")
                value = condition.get("value")

                # Apply conditions to filter the dataframe                
                if comparison == "eq":
                    null_check_df = null_check_df.where(F.col(column) == value)
                elif comparison == "ne":
                    null_check_df = null_check_df.where((F.col(column) != value) | (F.col(column).isNull()))
        
        for column in columns:
            # Define error message
            error_message = custom_error_message if custom_error_message else f"Column {column} cannot be null but contains a null value"        

            # Get rows which have a null value in the column
            errors_df = null_check_df.where(F.col(column).isNull())

            # Add error message column
            errors_df = errors_df.withColumn("error_message", F.lit(error_message))

            # Log the failed rows
            self._log_failed_rows(data_df=errors_df,
                                  validation=validation,
                                  rule_name=rule_name,
                                  validation_stage="field_validation")        

        return
    
    @add_try_except
    def _expect_column_values_to_not_be_null(self, null_check_df: DataFrame, columns: list[str], custom_error_message: str, rule_name: str, conditions: list[dict]) -> None:
        """Validates if a column is not null.
        Validate if a column is not NULL. If any column in the columns list is Null, flag it as an 
        error and add the custom_error_message. If supplied, the optional conditions parameter is 
        used to filter the dataframe before validation, ensuring the check is only applied to a 
        specific subset of data.

        Args:
            null_check_df (DataFrame): Spark DataFrame to validate
            columns (list[str]): List of column(s) in the DataFrame to be validated
            custom_error_message (str): Custom error message to use.
            rule_name (str): Name of the metadata rule that called this function
            conditions (list[dict]): List of dictionaries which define conditions on which to apply the check.

        Returns:
            None
        """
        validation = "Expect Not Null"

        if conditions:
            for condition in conditions:
                column = condition.get("column")
                comparison = condition.get("comparison")
                value = condition.get("value")
                
                if comparison == "eq":
                    null_check_df = null_check_df.where(F.col(column) == value)
                elif comparison == "ne":
                    null_check_df = null_check_df.where((F.col(column) != value) | (F.col(column).isNull()))
        
        for column in columns:
            error_message = custom_error_message if custom_error_message else f"Column {column} cannot be null but contains a null value"        

            errors_df = null_check_df.where(F.col(column).isNull())
            errors_df = errors_df.withColumn("error_message", F.lit(error_message))
            self._log_failed_rows(data_df=errors_df,
                                  validation=validation,
                                  rule_name=rule_name,
                                  validation_stage="field_validation")

        return

    @add_try_except
    def _expect_at_least_one_non_null_column(self, one_non_null_df: DataFrame, columns: list, custom_error_message: str, rule_name:str, conditions: list[dict] = None) -> None:
        """Validates if ALL columns in the columns list are null.
        At least one of the columns in the list MUST have a value. If supplied, the 
        optional conditions parameter is used to filter the dataframe before validation, 
        ensuring the check is only applied to a specific subset of data.

        Args:
            one_non_null_df (DataFrame) : Dataframe to validate.
            columns (list)              : List of columns to check for Nulls.
            custom_error_message (str)  : Custom error message. This string has 'column_placeholder' in it which 
                                            will be replaced by the current column that is being validated.
            rule_name (str)             : Name of the metadata rule that called this function.
            conditions (list[dict])     : List of dictionaries which define conditions on which to apply the check.

        Returns:
            None
        """
        validation="Expect One Non Null"

        if conditions:
            for condition in conditions:
                column = condition.get("column")
                comparison = condition.get("comparison")
                value = condition.get("value")
                
                if comparison == "eq":
                    one_non_null_df = one_non_null_df.where(F.lower(F.col(column)) == value.lower())
                elif comparison == "ne":
                    one_non_null_df = one_non_null_df.where((F.lower(F.col(column)) != value.lower()) | (F.col(column).isNull()))
        
        error_message = custom_error_message if custom_error_message else f"The columns {columns} must contain at least one non null value. Populate at least one of these columns and resubmit"
        
        errored_df = (
            one_non_null_df
            .withColumn("Result", F.when(F.coalesce(*columns).isNull(), "Error").otherwise("Okay"))
            .where("Result == 'Error'")
        )

        errored_df = errored_df.withColumn("error_message", F.lit(error_message))

        self._log_failed_rows(data_df=errored_df,
                              validation=validation,
                              rule_name=rule_name,
                              validation_stage="field_validation")
        
        return

    @add_try_except    
    def _expect_relative_date(self, source_data_df: DataFrame, columns: list, custom_error_message: str, time_check_type: str, rule_name:str) -> None:
        """Function to validate if a given date is today or earlier, or that a date is in the future.

        Args:
            source_data_df (DataFrame): Dataframe to validate.
            columns (list): Columns to validate.
            custom_error_message (str): Custom error message. This string has 'column_placeholder' in it which 
                                        will be replaced by the current column that is being validated.
            time_check_type (str): If time is today_or_earlier then that means all values in columns <= today. 
                                   If time is today_or_later that means all values in columns >= today. If the column is Null, ignore it.
            rule_name (str): Name of the metadata rule which called this function.
        
        Returns:
            None
        """
        validation = "Expect Relative Date"
        for column in columns:
            error_message = custom_error_message if custom_error_message else f"The column {column} must contain a date that is {time_check_type}"

            if time_check_type == "today_or_earlier":
                errored_df = (
                    source_data_df
                    .withColumn("Result", F.when(
                                            F.col(column).isNull(), "Okay").otherwise(
                                                F.when(
                                                    F.to_date(column, "dd/MM/yyyy") <= F.current_date(), "Okay"
                                                ).otherwise("Error")
                                            ))
                    .where("Result == 'Error'")
                )
            elif time_check_type == "today_or_later":
                errored_df = (
                    source_data_df
                    .withColumn("Result", F.when(
                                            F.col(column).isNull(), "Okay").otherwise(
                                                F.when(
                                                    F.to_date(column, "dd/MM/yyyy") >= F.current_date(), "Okay"
                                                ).otherwise("Error")
                                            ))
                    .where("Result == 'Error'")
                )
                        
            errored_df = errored_df.withColumn("error_message", F.lit(error_message))
            self._log_failed_rows(data_df=errored_df,
                                  validation=validation,
                                  rule_name=rule_name,
                                  validation_stage="field_validation")
        
        return
    
    @add_try_except
    def _expect_format(self, source_data_df: DataFrame, columns: list, format: str, length: int, rule_name:str, custom_error_message: str) -> None:
            """Function to validate the format of a given column.
            At the moment only possible format check is length of strings.

            Args:
                source_data_df (DataFrame): Dataframe to validate.
                columns (list): Columns to validate.
                custom_error_message (str): Custom error message. This string has 'column_placeholder' in it which 
                                            will be replaced by the current column that is being validated.
                format (str): Type of format we are validating. Only 'string' is accepted for now.
                length (int): Expected length of the string.
                rule_name (str): Name of the metadata rule that called this function

            Returns:
                None
            """
            validation = "Expect Format"

            for column in columns:
                error_message = custom_error_message if custom_error_message else f"The values in column {column} must be of length {length}."
                
                # errors are rows where value is not blank and value length is not expected length
                errored_df = source_data_df.where((F.length(F.col(column)) != length) & (F.col(column) != "") & (F.col(column).isNotNull()))
                errored_df = errored_df.withColumn("error_message", F.lit(error_message))
                self._log_failed_rows(data_df=errored_df,
                                      validation=validation,
                                      rule_name=rule_name,
                                      validation_stage="field_validation")                
            
            return
    
    @add_try_except
    def _column_length_checks(self, source_data_df: DataFrame, length_check_validations: list, rule_name:str) -> None:
        """Function to validate the length of a given column. If the column to validate does not exist in the dataframe,
        then it is skipped and the column name is send to logger. If the column to validate is null, then it is skipped.

        Args:
            source_data_df (DataFrame): Dataframe to validate.
            length_check_validations (list): List which contains column names and their corresponding max allowed length.
            rule_name (str): Name of the metadata rule that called this function.
            E.g:
                [   
                    {'name': 'GFRefNum', 'max_length': 200},
                    {'name': 'FName', 'max_length': 200},
                    {'name': 'SName', 'max_length': 200}
                ]
        """
        validation = "Column Length"
        
        for item in length_check_validations:
            column = item.get("name")
            max_length = item.get("max_length")

            errored_df = (
                source_data_df
                .withColumn(
                    "Result", F.when(
                        F.col(column).isNull(), "Okay").otherwise(
                            F.when(
                                F.length(F.col(column)) <= max_length,
                                "Pass"
                            ).otherwise("Error")
                        )
                    )
                .where("Result == 'Error'")
            )

            error_message = f"The length of the column {column} exceeds max length of {max_length}"
            errored_df = errored_df.withColumn("error_message", F.lit(error_message))
            self._log_failed_rows(data_df=errored_df,
                                  validation=validation,
                                  rule_name=rule_name,
                                  validation_stage="field_validation")        

        return

    # Row processing functions
    @add_try_except
    @log_method_call("row_processing")
    def row_processing(self, source_data_df: DataFrame, metadata_rules: dict) -> DataFrame:
        """Main function to do row processing for a given entity.

        Args:
            source_data_df (DataFrame): Source data frame.
            metadata_rules (dict): Dictionary containing rules to do the row processing.

        Returns:
            Dataframe after all row processing functions have been called.
        """
        
        for rule in metadata_rules:
            processing_type = rule.get("type")
            rule_name = rule.get("name")

            match processing_type:
                case "default":
                    columns = rule.get("columns")
                    value = rule.get("value")
                    suffix = rule.get("suffix")

                    source_data_df = self._set_default_column_value(data_df=source_data_df,
                                                                    columns=columns,
                                                                    default_value=value,
                                                                    suffix=suffix,
                                                                    rule_name=rule_name)

                case "copy_columns":
                    columns_to_copy = rule.get("copy_columns")

                    source_data_df = self._copy_columns(source_data_df=source_data_df,
                                                        columns=columns_to_copy)

                case "concat":
                    columns_to_concat = rule.get("columns")
                    output = rule.get("output")
                    separator = rule.get("separator", "")

                    source_data_df = self._concat_column_values(source_data_df=source_data_df,
                                                                columns=columns_to_concat,
                                                                output_column=output,
                                                                sep=separator,
                                                                rule_name=rule_name)

                case "format_date":
                    conditions=rule.get("conditions")
                    suffix = rule.get("suffix")
                    format_pattern = rule.get("format")
                    columns = rule.get("columns")
                    custom_error_code = rule.get("custom_error_code", None)
    
                    source_data_df = self._format_data(source_data_df=source_data_df,
                                                       format_name="date",
                                                       format_pattern=format_pattern,
                                                       columns=columns,
                                                       suffix=suffix,
                                                       conditions=conditions,
                                                       error_message=custom_error_code,
                                                       rule_name=rule_name)
                    
                case "format_datetime":                    
                    conditions=rule.get("conditions")
                    suffix = rule.get("suffix")
                    format_pattern = rule.get("format")
                    columns = rule.get("columns")
                    custom_error_code = rule.get("custom_error_code", None)

                    source_data_df = self._format_data(source_data_df=source_data_df,
                                                       format_name="datetime",
                                                       format_pattern=format_pattern,
                                                       columns=columns,
                                                       suffix=suffix,
                                                       conditions=conditions,
                                                       error_message=custom_error_code,
                                                       rule_name=rule_name)

                case "format_post_code":
                    conditions=rule.get("conditions")
                    suffix = rule.get("suffix")
                    format_pattern = None
                    columns = rule.get("columns")
                    custom_error_code = rule.get("custom_error_code", None)
    
                    source_data_df = self._format_data(source_data_df=source_data_df,
                                                       format_name="postcode",
                                                       format_pattern=format_pattern,
                                                       columns=columns,
                                                       suffix=suffix,
                                                       conditions=conditions,
                                                       error_message=custom_error_code,
                                                       rule_name=rule_name)

                case "format_email_address":
                    conditions=rule.get("conditions")
                    suffix = rule.get("suffix")
                    format_pattern = None
                    columns = rule.get("columns")
                    custom_error_code = rule.get("custom_error_code", None)

                    source_data_df = self._format_data(source_data_df=source_data_df,
                                                       format_name="email",
                                                       format_pattern=format_pattern,
                                                       columns=columns,
                                                       suffix=suffix,
                                                       conditions=conditions,
                                                       error_message=custom_error_code,
                                                       rule_name=rule_name)

                case "format_phone_number":
                    conditions = rule.get("conditions")
                    suffix = rule.get("suffix")
                    format_pattern = None
                    columns = rule.get("columns")
                    custom_error_code = rule.get("custom_error_code", None)
    
                    source_data_df = self._format_data(source_data_df=source_data_df,
                                                       format_name="phone",
                                                       format_pattern=format_pattern,
                                                       columns=columns,
                                                       suffix=suffix,
                                                       conditions=conditions,
                                                       error_message=custom_error_code,
                                                       rule_name=rule_name)

                case "date_part":
                    column = rule.get("column")
                    date_part = rule.get("date_part")
                    output_column_name = rule.get("output")
                    custom_error_code = rule.get("custom_error_code", None)
    
                    source_data_df = self._add_date_part(source_data_df=source_data_df,
                                                         date_column=column,
                                                         date_part=date_part,
                                                         output_column=output_column_name,
                                                         error_message=custom_error_code,
                                                         rule_name=rule_name)

                case "remove_characters":
                    column_to_clean = rule.get("column")
                    characters_to_remove = rule.get("characters")
                    output_column_name = rule.get("output")
    
                    source_data_df = self._remove_characters(data_df=source_data_df,
                                                             characters=characters_to_remove,
                                                             column_name=column_to_clean,
                                                             output_column=output_column_name,
                                                             rule_name=rule_name)

                case "backdate_four_tax_years":
                    column_to_backdate = rule.get("date")
                    output_column = rule.get("output")
    
                    source_data_df = self._backdate_four_tax_years(data_df=source_data_df,
                                                                   backdate_column=column_to_backdate,
                                                                   output_column_name=output_column,
                                                                   rule_name=rule_name)


                case "convert_values":
                    columns_to_convert = rule.get("columns")
                    replaces = rule.get("replaces")
                    suffix = rule.get("suffix")
                    custom_error_code = rule.get("custom_error_code", None)
                    
                    behaviour = rule.get("behaviour", None)
        
                    source_data_df = self._convert_values(source_data_df=source_data_df,
                                                          columns=columns_to_convert,
                                                          replaces=replaces,
                                                          suffix=suffix,
                                                          error_message=custom_error_code,
                                                          rule_name=rule_name,
                                                          behaviour=behaviour)
        
                case "create_field":
                    value = rule.get("value", None)
                    function = rule.get("function", None)
                    output = rule.get("output")
                    conditions = rule.get("conditions", None)

                    source_data_df = self._create_field(source_data_df=source_data_df,
                                                        value=value,
                                                        function=function,
                                                        output=output,
                                                        rule_name=rule_name,
                                                        conditions=conditions) 
                
                case "first_non_null":
                    columns = rule.get("columns")
                    output = rule.get("output")
                    source_data_df = self._first_non_null(source_data_df=source_data_df,
                                                          columns=columns,
                                                          output=output,
                                                          rule_name=rule_name)
                
                case "create_conditional_column":
                    if_thens = rule.get("if_thens")
                    else_value = rule.get("else")
                    output = rule.get("output")

                    source_data_df = self._create_conditional_column(source_data_df=source_data_df,
                                                                     if_thens=if_thens,
                                                                     else_value=else_value,
                                                                     output=output,
                                                                     rule_name=rule_name)
                case _:
                    if self._verbose_logging:
                        self._logger.warning(f"Unknown processing type: {processing_type} in rule: {rule_name}")
        

        return source_data_df
    
        
    def _format_data(self, source_data_df: DataFrame, format_name: str, format_pattern: str, columns: list[str], suffix:str, conditions: list[str], error_message: str, rule_name:str) -> DataFrame:
        """Checks a data source meets the expected format.
        The function checks the format_name parameter and performs transformation
        and validation operations. If any rows in the supplied Dataframe do not
        pass validation then the errors are logged.

        Args:
            source_data_df (DataFrame): Spark dataset containing the rows to validate.
            format_name (str): The format to validate - must be one of email/phone/postcode/date.
            format_pattern (str): An option expected format pattern (used for date).
            columns (list[str]): A list of columns to apply this formatting to, e.g a list of email addresses.
            suffix (str): The suffix to use for the transformed column name.
            conditions (list[str]): Conditions under which to apply formatting, e.g where email address != "anonymous"
            error_message (str): An optional custom error message which is only used if provided.
            rule_name (str): Name of the metadata rule that called this function
        
        Returns:
            formatted_df (DataFrame): The transformed dataframe.
        """
        formatted_df = source_data_df

        match (format_name):
            case "email":
                validation = "Format Email address"

                # Regular expression pattern for standard email address
                email_regex = "^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"

                for column_name in columns:
                    # Get error message if no custom metadata provided
                    email_error_message = f"{column_name} must be in a standard email address format" if not error_message else error_message

                    condition_expr = None
                    if conditions:
                        for condition in conditions:
                            column = condition["column"]
                            comparison = condition["comparison"]
                            value = condition['value']

                            if comparison == "eq":
                                expr = F.col(column) == value
                            elif comparison == "ne":
                                expr = ((F.col(column) != value) | (F.col(column).isNull()) )
                            else:
                                if self._verbose_logging:
                                    self._logger.warning(f"Unsupported comparison operator: {comparison} in {rule_name}")

                            if condition_expr is None:
                                condition_expr = expr
                            else:
                                condition_expr = condition_expr & expr

                    # Drop the email address to lowercase and create a new column using the condition if applicable
                    if condition_expr is not None:
                        formatted_df = formatted_df.withColumn(f"{column_name}{suffix}", 
                            F.when(condition_expr, F.lower(column_name))
                            .otherwise(None))
                    else:
                        formatted_df = formatted_df.withColumn(f"{column_name}{suffix}", F.lower(column_name)) 

                    formatted_df = formatted_df.withColumn(
                        "error_message",
                        F.when(
                            ~F.col(f"{column_name}{suffix}").rlike(email_regex),
                            F.lit(email_error_message),
                        ).when(
                            F.col(f"{column_name}{suffix}").contains("test"),
                            "Email address indicates this is a test row",
                        )
                    )

                    formatted_df = self._log_failed_rows(data_df=formatted_df,
                                                         validation=validation,
                                                         rule_name=rule_name,
                                                         validation_stage="row processing")
                        
            case "phone":
                validation = "Format Phone"

                for column_name in columns:
                    formatted_column_name = f"{column_name.strip('`')}{suffix}"

                    # Get set error message if no custom metadata provided
                    phone_error_message = f"{column_name} must be in a standard telephone format" if not error_message else error_message

                    # In all outcomes of the operation remove '+' characters and whitespace
                    #  and if the formatted column value starts with 07 then replace 07 with 447
                    #  and if the column value starts with 7 then replace with 447
                    #  then log as an error message if the string value is non numeric
                    formatted_df = (
                        formatted_df
                        .withColumn(
                            formatted_column_name,
                            F.when(
                                F.col(column_name).startswith("07"),
                                F.expr(f"CONCAT('447', SUBSTRING(regexp_replace(`{column_name}`, '[ +]', ''), 3, LENGTH(`{column_name}`)))")
                            ).when(
                                F.col(column_name).startswith("7"),
                                F.expr(f"CONCAT('447', SUBSTRING(regexp_replace(`{column_name}`, '[ +]', ''), 2, LENGTH(`{column_name}`)))")  
                            ).otherwise(
                                F.regexp_replace(F.col(column_name), "[ +]", "")
                            ))
                        .withColumn(
                            "error_message",
                            F.when(
                                ~F.col(formatted_column_name).rlike("^[0-9]*$"),
                                F.lit(phone_error_message)
                            ).otherwise(
                                    F.lit(None)
                            ))
                        )
                    
                    formatted_df = self._log_failed_rows(data_df=formatted_df,
                                                         validation=validation,
                                                         rule_name=rule_name,
                                                         validation_stage="row processing")

            case "postcode":
                # Postcode formatting does not produce an error.
                validation = "Format Postcode"

                # Regular expression pattern for standard email address
                postcode_regex = "^[A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][ABD-HJLNP-UW-Z]{2}$"
                for column_name in columns:
                    # If the postcode conforms to the standard UK format then set the value as
                    #  uppercase and add a space prior to the last three characters, but if the postcode
                    #  does not conform to the expected format then make no transformations
                    formatted_df = formatted_df.withColumn(
                        "temp_postcode",
                        F.expr(f"CONCAT(UPPER(SUBSTRING(regexp_replace(`{column_name}`, ' ', ''), 1, LENGTH(regexp_replace(`{column_name}`, ' ', ''))-3)), ' ', UPPER(SUBSTRING(regexp_replace(`{column_name}`, ' ', ''), LENGTH(regexp_replace(`{column_name}`, ' ', ''))-2, 3)))")
                    ).withColumn(
                        f"{column_name}{suffix}",
                        F.when(
                            ~F.col("temp_postcode").rlike(postcode_regex),
                            F.col(column_name)
                        ).otherwise(
                            F.col("temp_postcode")
                        )
                    ).drop("temp_postcode")

            case "date":
                validation = "Format Date"

                for column_name in columns:
                    # Get set error message if no custom metadata provided
                    if format_pattern != 'any':
                        date_error_message = f"{column_name} must be in date format: {format_pattern}" if not error_message else error_message
                    else:
                        date_error_message = f"{column_name} must be in date format yyyy-MM-dd HH:mm:ss" if not error_message else error_message

                    formatted_column_name = f"{column_name.strip('`')}{suffix}"

                    if format_pattern != "any":
                        # Try to parse the date directly - let PySpark handle validation
                        parsed_date = F.to_date(F.col(column_name), format_pattern)

                    else:
                        valid_formats = [
                            "dd/MM/yyyy HH:mm:ss",
                            "dd/MM/yyyy HH:mm",
                            "dd/MM/yy", 
                            "dd/MM/yyyy",
                            "yyyy-MM-dd HH:mm:ss",
                            "yyyy-MM-dd HH:mm"
                        ]

                        # loop through valid formats
                        parsed_date_attempts = [
                            F.to_date(F.col(column_name), fmt) for fmt in valid_formats
                        ]

                        # use coalesce to return the first successful format
                        parsed_date = F.coalesce(*parsed_date_attempts)

                    formatted_df = (
                        formatted_df
                        .withColumn(
                            formatted_column_name,
                            F.when(
                                (F.col(column_name).isNotNull()) & 
                                (F.col(column_name) != "") &
                                (parsed_date.isNotNull()),
                                parsed_date
                            ).otherwise(F.lit(None))
                        )
                        .withColumn(
                            "error_message",
                            F.when(
                                (F.col(column_name).isNotNull()) & 
                                (F.col(column_name) != "") &
                                (parsed_date.isNull()),
                                F.lit(date_error_message)
                            ).otherwise(F.lit(None))
                        )
                    )
        
                    formatted_df = self._log_failed_rows(data_df=formatted_df,
                                                         validation=validation,
                                                         rule_name=rule_name,
                                                         validation_stage="row processing")

            case "datetime":
                validation = "Format Datetime"

                for column_name in columns:
                    if format_pattern != "any":
                        # Get set error message if no custom metadata provided
                        datetime_error_message = f"{column_name} must be in date format: {format_pattern}" if not error_message else error_message
                    else:
                        datetime_error_message = f"{column_name} must be in date format yyyy-MM-dd HH:mm:ss" if not error_message else error_message
                    formatted_column_name = f"{column_name.strip('`')}{suffix}"

                    if format_pattern != "any":
                        # Try to parse the date directly - let PySpark handle validation
                        parsed_date = F.to_timestamp(F.col(column_name), format_pattern)

                    else:
                        valid_formats = [
                            "dd/MM/yyyy HH:mm:ss",
                            "dd/MM/yyyy HH:mm",
                            "dd/MM/yy", 
                            "dd/MM/yyyy",
                            "yyyy-MM-dd HH:mm:ss",
                            "yyyy-MM-dd HH:mm"
                        ]

                        # loop through valid formats
                        parsed_date_attempts = [
                            F.to_timestamp(F.col(column_name), fmt) for fmt in valid_formats
                        ]

                        # use coalesce to return the first successful format
                        parsed_date = F.coalesce(*parsed_date_attempts)

                    formatted_df = (
                        formatted_df
                        .withColumn(
                            formatted_column_name,
                            F.when(
                                (F.col(column_name).isNotNull()) & 
                                (F.col(column_name) != "") &
                                (parsed_date.isNotNull()),
                                parsed_date
                            ).otherwise(F.lit(None))
                        )
                        .withColumn(
                            "error_message",
                            F.when(
                                (F.col(column_name).isNotNull()) & 
                                (F.col(column_name) != "") &
                                (parsed_date.isNull()),
                                F.lit(datetime_error_message)
                            ).otherwise(F.lit(None))
                        )
                    )

                    formatted_df = self._log_failed_rows(data_df=formatted_df,
                                                         validation=validation,
                                                         rule_name=rule_name,
                                                         validation_stage="row processing")

            case _:
                if self._verbose_logging:
                    self._logger.warning(f"Received unexpected format '{format_name}' - must be one of email/phone/postcode/date/datetime")                    
                
        return formatted_df
    
    def _concat_column_values(self, source_data_df: DataFrame, columns: list[str], output_column: str, sep: str, rule_name:str) -> DataFrame:
        """Concatenates column values together into a new column.
        Accepts a dataframe, a list of strings and a separator character and 
        creates a new column by concatenating the values of the supplied columns.
        The values are separated by the optional separator character and the new column 
        is named from the output_column parameter. If all the input columns are null, a null is returned

        Args:
            source_data_df (DataFrame): Spark dataframe containing the ingested file.
            columns (list[str]): List of strings containing required column names.
            output_column (str): Name of the column to create.
            sep (str): Separator character to use when concatenating column values.
            rule_name (str): Name of the metadata rule that called this function
            
        Returns:
            concat_df (DataFrame): The dataframe with the additional derived column.
        """
        # Concatenate the column values with the supplied separator
        concat_df = source_data_df.withColumn(output_column, F.concat_ws(sep, *columns))
        
        # Replace output column with null if it is equal to empty string, such as when concatting all null columns
        concat_df = concat_df.withColumn(output_column, F.when(F.col(output_column) != "", F.col(output_column)).otherwise(F.lit(None)))

        return concat_df    

    def _copy_columns(self, source_data_df: DataFrame, columns: list[dict]) -> DataFrame:
        """Copies specified column values into new columns.
        Iterates a list of column names and extracts the values in the specified columns 
        into new derived columns with the required name.

        Args:
            source_data_df (DataFrame): Spark dataframe containing the ingested data.
            columns (list[dict]): A list of dictionaries containing the columns
                                    to extract from and new columns to create.
            
            Example columns value:
            [{
                "column": "FName",
                "new_column_name": "NewFName"
            },
            {
                "column": "SName",
                "new_column_name": "NewLName"
            }]
            rule_name (str): Name of the metadata rule that called this function
            
        Returns:
            new_cols_df (DataFrame): Transformed data including the new columns.
        """
        # Iterate the copy columns assigning the existing column value to the new name
        for item in columns:
            # Create a new column with the same values as the original column
            copied_cols_df = source_data_df.withColumn(item["new_column_name"], F.col(item["column"]))        

        return copied_cols_df

    def _set_default_column_value(self, data_df: DataFrame, columns: list[str], default_value: Any, suffix: str, rule_name:str) -> DataFrame:
        """Creates new defaulted column with values if null.
        For each column name specified in the columns list, a new column is 
        created in the supplied dataframe taking the value of the existing column 
        but if the value is null then the default_value supplied is used. The new
        column is named from the existing column name with the value of the suffix
        parameter appended to the string e.g. FirstName_defaulted.
        If the column does not exist in the dataset then create a new column column_suffix 
        with the value

        Args:
            data_df (DataFrame): Spark dataframe too add the columns to.
            columns (list[str]): List of column names to process.
            default_value (str): Default value to assign if none exists.
            rule_name (str): Name of the metadata rule that called this function
            
        Returns:
            data_df (DataFrame): Spark dataframe containing the default values.
        """
        for column_name in columns:
            if column_name in data_df.columns:
                data_df = (
                    data_df
                    .withColumn(f"{column_name}{suffix}",
                                F.when((F.col(column_name).isNull()) | (F.col(column_name) == ""), default_value)
                                .otherwise(F.col(column_name)))
                    )
            else:
                data_df = (
                    data_df
                    .withColumn(f"{column_name}{suffix}", F.lit(default_value))
                    )

        
        return data_df

    def _remove_characters(self, data_df: DataFrame, characters: list[str], column_name: str, output_column: str, rule_name:str) -> DataFrame:
        """Creates a copy of supplied columns removing required characters.
        Accepts a list of characters and column name and creates a new column
        named from the output_name parameter containing the existing contents
        of the column. Then proceeds to remove all occurances of characters in
        the characters parameter preserving the original value. If after removing
        all characters, only an empty string remains, a null is returned in that cell.

        Args:
            data_df (DataFrame): Spark dataframe to transform.
            characters (list[str]): List of characters to remove.
            column_name (str): Column name to transform.
            output_column (str): Name of the created column.
            rule_name (str): Name of the metadata rule that called this function

        Returns:
            data_df (DataFrame): Spark dataframe containing the transformed column.
        """
        # Create a new column with the same values as the original column  
        data_df = data_df.withColumn(output_column, F.col(column_name))  
    
        # Apply regexp_replace function to the new column in the loop
        for character in characters:
            data_df = data_df.withColumn(output_column,
                                         F.regexp_replace(F.col(output_column), character, "")
                                         )

        # After removing characters if only an empty string is remaining, this is set to null.
        data_df = data_df.withColumn(output_column,
                                     F.when(F.col(output_column) != "", F.col(output_column))
                                     .otherwise(F.lit(None))
                                     )

        return data_df
    
    def _backdate_four_tax_years(self, data_df: DataFrame, backdate_column:str, output_column_name: str, rule_name:str) -> DataFrame:
        """Backdates a given backdate_column to 1st April and subtract 4 years.

        Args:
            data_df (DataFrame) : Spark dataframe to transform.
            backdate_column (str) : Column to backdate. MUST be a date column.
            output_column_name (str) : Newly backdated column
            rule_name (str): Name of the metadata rule that called this function
            
        Returns:
            backdated_df (DataFrame) where the column has been backdated to 1st April four years ago.

        Eg:
            +---------------------+--------------+
            |Starting Date_as_date|backdated_date|
            +---------------------+--------------+
            |           2024-12-01|    2020-04-01|
            |           2025-05-01|    2021-04-01|
            |           2025-02-01|    2020-04-01|
            +---------------------+--------------+
        """
        # Calculate the tax year base year
        backdated_df = (
            data_df
            .withColumn("base_year", 
                F.when(F.month(F.col(backdate_column)) < 4, 
                    F.year(F.col(backdate_column)) - 5)
                .otherwise(F.year(F.col(backdate_column))-4)
            )
            .withColumn(output_column_name, F.concat_ws("-",
                F.col("base_year"), F.lit("04"), F.lit("01")))
        )

        return backdated_df
    
    def _add_date_part(self, source_data_df: DataFrame, date_column: str, date_part: str, output_column: str, error_message: str, rule_name:str) -> DataFrame:
        """Creates a new column by extracting the required date part from column values.

        Args:
            source_data_df (DataFrame): The dataframe to transform.
            date_column (str): The name of the date column to transform.
            date_part (str): The part of the date to extract (day/month/year)
            output_column (str): The name of the column to create.
            error_message (str): Optional custom error message.
            rule_name (str): Name of the metadata rule that called this function

        Returns:
            check_date_df (DataFrame): Transformed dataframe.
        """
        validation = "Date Part"
        
        if date_part == "year":
            check_date_df = (
                source_data_df
                .withColumn(output_column,
                    F.when(
                        F.col(date_column).isNotNull(),
                        F.year(F.col(date_column))
                    ).otherwise(F.lit(None))
                )
            )
        elif date_part == "month":
            check_date_df = (
                source_data_df
                .withColumn(output_column,
                    F.when(
                        F.col(date_column).isNotNull(),
                        F.month(F.col(date_column))
                    ).otherwise(F.lit(None))
                )
            )
        elif date_part == "day":
            check_date_df = (
                source_data_df
                .withColumn(output_column,
                    F.when(
                        F.col(date_column).isNotNull(),
                        F.dayofmonth(F.col(date_column))
                    ).otherwise(F.lit(None))
                )
            )
        else:
            if self._verbose_logging:
                self._logger.warning(f"unexpected date_part {date_part} was supplied")

        error_rows_df = check_date_df.where((F.col(output_column).isNull()) & (F.col(date_column).isNotNull()))

        self._log_failed_rows(error_rows_df, validation, rule_name, "row processing", error_message)


        return check_date_df  
 
    def _convert_values(self, source_data_df: DataFrame, columns: list, replaces: list[dict], suffix: str, error_message: str, rule_name:str, behaviour: str) -> DataFrame:
        """Creates copies of columns replacing existing values with supplied values.
        For each column in the list supplied, a new column is created named from the
        existing column name suffixed with the suffix parameter. The values in the
        new column are then replaced by the values in the replaces parameter. If the
        behaviour parameter is set to "keep_others" then any value not replaced will be
        retained as is, if the behaviour is "null_others" then values not replaced will
        be replaced by a null value, and if the behavious is "fail_row" which is the
        default then the error will be logged.
    
        Args:
            source_data_df (DataFrame): Spark dataframe of source data.
            columns (list): List of column names to perform the operation against.
            replaces (list[dict]) : List of dictionary of from and to values to convert.
            suffix (str): The suffix of the new column name.
            error_message (str): An optional custom error message.
            rule_name (str): Name of the metadata rule that called this function
            behaviour (str): Specifies how to handle values not replaced. Has a default value
                            of "keep_others" allowing for no value to be supplied and no 
                            action taken.
    
        Returns:
            source_data_df (DataFrame): Spark dataframe with new columns.
        """
        from_values = [key["from"] for key in replaces]
        validation = "Convert Values"
        
        if behaviour:
            behaviour = behaviour.get("if_other_values_found")
        else:
            behaviour = "keep_others"

        for column in columns:
            convert_error_message = error_message if error_message else f"{column} is blank or contains a value that is not allowed. Replace it with one of the following: " + ", ".join(from_values)

            new_column = f"{column}{suffix}"
            source_data_df = source_data_df.withColumn(new_column, F.lit(None))

            for replace in replaces:
                if behaviour == "keep_others":
                    source_data_df = (
                        source_data_df
                        .withColumn(new_column,
                                    F.when(
                                        (F.lower(F.col(column)) == F.lower(F.lit(replace["from"])))
                                        & (F.col(new_column).isNull()),
                                        replace["to"])
                                    .otherwise(F.col(new_column))
                                    )
                                )
                elif behaviour == "null_others":
                    source_data_df = (                    
                        source_data_df
                        .withColumn(new_column,
                                    F.when(F.lower(F.col(column)) == F.lower(F.lit(replace["from"])), replace["to"])
                                    .otherwise(F.col(new_column))
                                )
                            )
                elif behaviour == "fail_row":
                    source_data_df = (
                        source_data_df
                            .withColumn(new_column,
                                    F.when(F.lower(F.col(column)) == F.lower(F.lit(replace["from"])), replace["to"])
                                    .otherwise(F.col(new_column)) # Retain the original value if no match
                                )
                            )
            
            # If behaviour is 'null_others' then set any unaccepted value to null        
            if behaviour == "null_others":
                valid_values = [item["to"] for item in replaces]

                source_data_df = source_data_df.withColumn(new_column, 
                    F.when(F.col(new_column).isin(valid_values), F.col(new_column))
                    .otherwise(F.lit(None))
                    )
            elif behaviour == "keep_others":
                source_data_df = (
                    source_data_df
                    .withColumn(new_column,
                                F.when(F.col(new_column).isNull(), F.col(column))
                                .otherwise(F.col(new_column))
                                )
                            )                   
            elif behaviour == "fail_row":            
                valid_values = [item["to"] for item in replaces] 
                errors_df = source_data_df.where(~F.col(new_column).isin(valid_values) | F.col(new_column).isNull())
                errors_df = errors_df.withColumn("error_message", F.lit(convert_error_message))
                
                self._log_failed_rows(errors_df, validation, rule_name, "row processing")

        
        return source_data_df
    
    
    def _create_field(self, source_data_df: DataFrame, value:str, function:str, output:str, rule_name:str, conditions: list[str]) -> DataFrame:
        """
        Creates a column in the data frame with a value from value or a function
        Args:
            source_data_df: DataFrame: Spark dataframe containing source data
            value(str): The value to assign to the column
            function(str): The function from which to get a value to assign to a column. Valid values are get_date and get_timestamp
            output:str): The name of the column to create 
            rule_name(str): The name of the rule from metadata, used for logging if required
            conditions(list): An optional list of conditions for rows where the field should be created.
        Returns:
            source_data_df (DataFrame): Spark dataframe with new columns.
        """
        
        # Iterate over each of the conditions in condition to create an expression function
        condition_expr = None
        if conditions:
            for condition in conditions:
                column = condition["column"]
                comparison = condition["comparison"]
                con_value = condition['value']

                if comparison == "eq":
                    expr = F.col(column) == con_value
                elif comparison == "ne":
                    expr = ((F.col(column) != value) | (F.col(column).isNull()) )
                else:
                    if self._verbose_logging:
                        self._logger.warning(f"Unsupported comparison operator: {comparison} in {rule_name}")

                if condition_expr is None:
                    condition_expr = expr
                else:
                    condition_expr = condition_expr & expr

        # If there are no conditions, then set the Condition expression to a column expression that always
        # evaluates to True. That way the update will be applied to all rows
        if condition_expr is None:
            condition_expr = F.lit(True)

        # Define the function that will be called if it is on a function
        if function:
            match function:
                case "get_date": 
                    field_value = F.lit(datetime.now().date())
                case "get_timestamp":
                    field_value = F.lit(datetime.now())
                case "create_guid":
                    field_value = F.expr("uuid()")
                case _:
                    if self._verbose_logging:
                        self._logger.warning(f"Unsupported create_field operator: {function}")

        else:
            field_value = F.lit(value)

        
        # Create the output by returning a column with field_value when the conditions are all true, otherwise a null value.
        create_df = source_data_df.withColumn(
            output,
            F.when(condition_expr, field_value)
            .otherwise(None))

        return create_df
    
    def _first_non_null(self, source_data_df: DataFrame, columns:list, output:str, rule_name:str) -> DataFrame:
        """
        Gets the first value that is not null from a list of columns
        Args:
            source_data_df: DataFrame: Spark dataframe containing source data
            columns(list): A list of the columns 
            output:(str): The name of the column to create 
            rule_name(str): The name of the rule from metadata, used for logging if required
            
        Returns:
            source_data_df (DataFrame): Spark dataframe with new columns.
        """
        coalesce_df = source_data_df.withColumn(output, F.coalesce(*[F.when(F.col(column) != "", F.col(column)).otherwise(None) for column in columns]))
        

        return coalesce_df 
    
    def _create_conditional_column(self, source_data_df: DataFrame, if_thens: list[dict], else_value: str, output: str, rule_name: str) -> DataFrame:
        """Creates a column with values depending on the supplied conditions.

        Args:
            source_data_df: DataFrame: Spark dataframe containing source data
            if_thens(list[dict]): A list of the conditions held in a dictionary with column, comparison, and value fields.
                                e.g. 
                                        "if_thens": [
                                            {
                                                "conditions": [
                                                {
                                                    "column": "Donor FirstName",
                                                    "comparison": "ne",
                                                    "value": "Anonymous"
                                                },
                                                {
                                                    "column": "Donation Consent",
                                                    "comparison": "eq",
                                                    "value": "Yes"
                                                }
                                            ],
                                            "then": true
                                            }
                                        ],
            else_value: The value to assign if the conditions are not met.
            output:(str): The name of the column to create .
            rule_name(str): The name of the rule from metadata, used for logging if required.

        Returns:
            source_data_df (DataFrame): Spark dataframe with new column.
        """
        conditional_df = source_data_df.withColumn(output, F.lit(None))

        expr = None
        for if_then in if_thens:
            then_value = if_then.get("then")
            then_column = if_then.get("then_source")
            conditions = if_then.get("conditions", [])
            cond_expr = None

            # Iterate the conditions to construct a PySpark filter expression
            for condition in conditions:
                column = condition["column"]
                comparison = condition["comparison"]
                value = condition.get("value")
                if comparison == "eq":
                    this_expr = F.col(column) == value
                elif comparison == "ne":
                    this_expr = ((F.col(column) != value) | (F.col(column).isNull()) )
                else:
                    this_expr = F.lit(False)
                    if self._verbose_logging:
                        self._logger.warning(f"Unsupported comparison operator: {comparison} in {rule_name}")
                
                cond_expr = this_expr if cond_expr is None else (cond_expr & this_expr)

            # Determine the value to use: either a literal or a column reference
            if then_column:
                value_expr = F.col(then_column)
            else:
                value_expr = F.lit(then_value)

            # Append the current expression to the when chain
            expr = F.when(cond_expr, value_expr) if expr is None else expr.when(cond_expr, value_expr)
        
        # Add an otherwise clause to apply if the conditions are not met
        expr = expr.otherwise(else_value)

        conditional_df = conditional_df.withColumn(output, expr)

        return conditional_df
    
    def _remove_whitespace(self, source_data_df: DataFrame, column: str, how: str) -> DataFrame:
        """Removes whitespace from a given column in a DataFrame.
        The function removes whitespace from the column specified in the column parameter.
        The how parameter determines whether whitespace is removed from both ends of the string,
        just the left end, or just the right end.

        Args:
            source_data_df (DataFrame): The DataFrame containing the column to process.
            column (str): The name of the column from which to remove whitespace.
            how (str): Specifies how to remove whitespace. Options are 'both', 'left', or 'right'.

        Returns:
            DataFrame: The DataFrame with whitespace removed from the specified column.
        """
        match how:
            case "both":
                trimmed_df = source_data_df.withColumn(column, F.trim(F.col(column)))
            case "left":
                trimmed_df = source_data_df.withColumn(column, F.ltrim(F.col(column)))
            case "right":
                trimmed_df = source_data_df.withColumn(column, F.rtrim(F.col(column)))
            case _:
                if self._verbose_logging:
                    self._logger.warning(f"Unsupported whitespace removal option: {how}. No changes made to column {column}.")
                trimmed_df = source_data_df

        return trimmed_df

    # Custom validation
    def custom_validation(self, source_data_df: DataFrame, metadata_rules: dict) -> bool:
        """Performs custom validations on a given DataFrame based on provided metadata rules.
        The supplied dictionary 'metadata_rules' should contain a 'rules' key which 
        is a list of dictionaries where each dictionary represents a validation rule. The rules 
        are iterated sequentially and separate validation functions called to validate different 
        requirements. If a file is deemed invalid then the function breaks out of the iteration and
        returns a boolean of true or false along with the supplied dataframe.

        Each validation rule should contain 'type', 'name' and optionally 'custom_error_code'.
        'type' is the type of validation rule and could be 'expect_column_values_to_be_unique_in_file', 
        'expect_column_values_to_not_be_null_custom' or 'expect_column_values'. 'name' is the name of the 
        validation rule. 'custom_error_code' is the custom error message for the validation rule.

        Args:
            source_data_df (pandas.DataFrame): The DataFrame on which the validations are to be performed.
            metadata_rules (dict): A dictionary containing validation rules.
        
        Returns:
            is_file_valid (bool): True if the DataFrame is valid based on the validation rules, False otherwise.
            source_data_df (DataFrame): The DataFrame after the validations have been performed.
        """       
        is_file_valid = True
        
        if metadata_rules is None:
            return is_file_valid, source_data_df

        rules = metadata_rules.get("rules")

        for rule in rules:
            validation_type = rule.get("type")
            rule_name = rule.get("name")
            custom_error_code = rule.get("custom_error_code", None)
            
            match validation_type:
                case "expect_column_values_to_be_unique_in_file":
                    column_name = rule.get("column")
                    action = rule.get("action")
                    error_message = f"{column_name} contains duplicate values" if not custom_error_code else custom_error_code

                    is_file_valid = self._expect_column_values_to_be_unique_in_file(source_data_df,
                                                                                    column_name,
                                                                                    action,
                                                                                    error_message,
                                                                                    rule_name)
                    
                case "expect_column_values_to_not_be_null_custom":
                    columns = rule.get("columns")
                    conditions = rule.get("conditions")
                    action = rule.get("action")
                    default_custom_error_code = rule.get("default_custom_error_code")
                    is_file_valid = self._expect_column_values_to_not_be_null_custom(source_data_df=source_data_df,
                                                                                     columns=columns,
                                                                                     conditions=conditions,
                                                                                     default_error_code=default_custom_error_code,
                                                                                     rule_name=rule_name,
                                                                                     action=action)
                    
                case "expect_column_values":
                    conditions = rule.get("conditions")
                    action = rule.get("action")
                    rule_name = rule.get("name")

                    source_data_df, is_file_valid = self._expect_column_values(source_data_df=source_data_df,
                                                                               conditions=conditions,
                                                                               action=action,
                                                                               rule_name=rule_name)
                    
                case "_":
                    print(f"Received unexpected validation type '{validation_type}'")
            
            # Exit the loop if the file is not valid
            if not is_file_valid:
                return is_file_valid, source_data_df


        return is_file_valid, source_data_df
    
    def _expect_column_values_to_be_unique_in_file(self, source_data_df: DataFrame, column_name: str, action: str, error_message: str, rule_name:str) -> bool:
        """Checks a data source does not have duplicates.
        The function checks for duplicates in the values in the column_name parameter. 
        If any rows in the supplied Dataframe have duplicate data then the errors are logged for each column.

        Args:
            source_data_df (DataFrame): Spark dataset containing the rows to validate.
            column_name (str): The name of the column to validate.
            action: The action to take in the event of duplicates. Options expected reject_file or reject_row. Default is reject_row.
            error_message (str): An optional custom error message which is only used if provided.
            rule_name (str): Name of the metadata rule that called this function.

        Returns:
            bool: Returns True or False to denote the validity of the file.
        """    
        duplicates_df = source_data_df.groupBy(column_name).count().where(F.col("count") > 1)
        validation = "Unique Value"

        if not duplicates_df.isEmpty():
            # Set an error message if error not provided
            if not error_message:
                error_message = "Duplicate value in " + column_name

            # Handle errors in file or row
            if action == "reject_file":
                return False
            else:
                # Add duplicate rows to a failed data frame
                failed_rows_df = source_data_df.join(duplicates_df, on=source_data_df[column_name] == duplicates_df[column_name], how="inner")
                failed_rows_df = failed_rows_df.withColumn("error_message", F.lit(error_message))
                self._log_failed_rows(failed_rows_df, validation, rule_name, "custom validation")
        
        # Return true to mark the file as not rejected
        return True
    
    def _expect_column_values_to_not_be_null_custom(self, source_data_df: DataFrame, columns: list, conditions:list[dict], default_error_code: str, rule_name: str, action: str) -> bool:
        """Checks that the column value is not null, and supplies a specific error message depending on values in other fields
        Args:
            source_data_df (DataFrame): Spark dataset containing the rows to validate.
            columns (list): The name of the columns to validate.
            action: The action to take in the event of duplicates. Options expected reject_file or reject_row. Default is reject_row
            error_message (str): An optional custom error message which is only used if provided.
            rule_name (str): Name of the metadata rule that called this function.

        Returns:
            bool
        """    
        validation = "Expect Not Null"
        errors_df = self._spark.createDataFrame([], schema=source_data_df.schema)
        
        for column in columns:
            error_df = source_data_df.where(F.col(column).isNull())
            errors_df = errors_df.union(error_df)
            
        if not errors_df.isEmpty():
            errors_df = errors_df.withColumn("Error", F.lit(None))

            for condition in conditions:
                condition_expr = None
                
                column = condition["column"]
                comparison = condition["comparison"]
                value = condition["value"]
                custom_error_code = condition["custom_error_code"]

                if comparison == "eq":
                    expr = F.col(column) == value
                elif comparison == "ne":
                    expr = ((F.col(column) != value) | (F.col(column).isNull()) )
                else:
                    self._logger.warning(f"Unsupported comparison operator: {comparison} in {rule_name}")

                condition_expr = expr            
                errors_df = errors_df.withColumn("Error", F.when(condition_expr, custom_error_code).otherwise(F.col("Error")))
            
            errors_df = errors_df.withColumn("Error", F.when(F.col("Error").isNull(), default_error_code).otherwise(F.col("Error")))

            if action == "reject_row":
                self._log_failed_rows(errors_df, validation, rule_name, "custom validation")
            elif action == "reject_file":
                self._log_failed_rows(errors_df, validation, rule_name, "custom validation")
                return False

        
        return True
    
    def _expect_column_values(self, source_data_df: DataFrame, conditions: list[dict], action: str, rule_name: str) -> DataFrame:
        """Conditionally performs actions such as dropping rows.
        The function accepts values parsed from metadata detailing conditional operations 
        to perform against the given dataframe. The specified action is only applied to 
        rows which meet the condition.

        Args:
            source_data_df (DataFrame): The DataFrame on which the operations are to be performed.
            conditions (list[dict]): A list of dictionaries where each dictionary represents a condition. Each dictionary 
                                    should contain 'column', 'comparison' and 'value' keys. 'column' is the column in the DataFrame 
                                    where the condition is to be applied. 'comparison' is the comparison operator and could be 'eq' (equal),
                                    'ne' (not equal), 'lt' (less than), 'gt' (greater than), 'isnull' (is null), 'notnull' (is not null). 
                                    'value' is the value for the comparison.
            action (str): The action to be performed if the condition is met. Currently, only 'drop_row' action is supported which
                        drops the row if the condition is met.
            rule_name (str): The name of the rule extracted from the metadata.
        
        Returns:
            source_data_df (DataFrame): Processed dataframe.
        """
        for condition in conditions:
            column = condition.get("column")
            comparison = condition.get("comparison")
            value = condition.get("value")
            
            if comparison == "eq":
                filter_condition = F.col(column) == value
                comparison_name = "equal"
            elif comparison == "ne":
                filter_condition = ((F.col(column) != value) | (F.col(column).isNull()) )
                comparison_name = "different"
            elif comparison == "lt":
                filter_condition = F.col(column) < value
                comparison_name = "less than"
            elif comparison == "gt":
                filter_condition = F.col(column) > value
                comparison_name = "greater than"
            elif comparison == "isnull":
                filter_condition = F.col(column).isNull()
                comparison_name = "null"
            elif comparison == "notnull":
                filter_condition = F.col(column).isNotNull()
                comparison_name = "not null"
            else:
                print(f"Recieved unexpected comparison operator of '{comparison}'")
            
            error_message = f"Column '{column}' should be {comparison_name} to '{value}'"
            
            if action == "drop_row":
                rowcount = source_data_df.count()
                
                # Apply filter
                filtered_df = source_data_df.where(filter_condition)

                # Remove filtered_df rows from source_data_df
                source_data_df = source_data_df.join(filtered_df, on=["row_id"], how="left_anti")

                new_rowcount = source_data_df.count()
                dropped_rowcount = rowcount - new_rowcount
            elif action == "fail_file":
                filtered_df = source_data_df.where(filter_condition)
                errors_df = filtered_df.withColumn("error_message", F.lit(error_message))
                self._log_failed_rows(errors_df, "expect_column_values", rule_name, "custom validation")

                return source_data_df, False
            elif action == "reject_row":
                filtered_df = source_data_df.where(filter_condition)
                errors_df = filtered_df.withColumn("error_message", F.lit(error_message))
                self._log_failed_rows(errors_df, "expect_column_values", rule_name, "custom validation")
            else:
                print(f"Recieved unexpected action operator of '{action}'")

                return source_data_df, False

        return source_data_df, True